# SmartEye v0.1 - OCR 및 Vision AI 시스템

이 노트북은 학습지 이미지의 레이아웃 분석과 시각 정보 처리를 위한 통합 시스템입니다.

## 주요 기능
1. **LAM (Layout Analysis Module)**: DocLayout-YOLO를 활용한 레이아웃 분석
2. **TSPM (Text & Semantic Processing Module)**: OCR 및 OpenAI Vision API를 통한 텍스트/이미지 처리
3. **CIM (Content Integration Module)**: 결과 통합 및 출력

## 지원 환경
- Google Colab / 로컬 환경 자동 감지
- Python 3.7+

## 1. 환경 감지 및 패키지 설치

In [ ]:
# ============================================
# 📦 환경 감지 및 패키지 설치
# ============================================

import os
import sys
import subprocess

def detect_environment():
    """현재 실행 환경을 감지합니다."""
    try:
        import google.colab
        return 'colab'
    except ImportError:
        return 'local'

def install_packages():
    """환경별 필수 패키지 설치"""
    env = detect_environment()
    print(f"🔧 환경: {env.upper()}")
    
    if env == 'colab':
        # Colab 환경용 설치
        print("🚀 Colab 패키지 설치 중...")
        !pip install -q numpy opencv-python matplotlib pillow tqdm PyMuPDF psutil pytesseract requests loguru huggingface_hub openai
        !apt-get update -qq && apt-get install -qq -y tesseract-ocr tesseract-ocr-kor fonts-nanum
        
        # DocLayout-YOLO 설치
        if not os.path.exists('DocLayout-YOLO'):
            !git clone https://github.com/opendatalab/DocLayout-YOLO.git
            %cd DocLayout-YOLO
            !pip install -q -e .
            %cd ..
            
    else:
        # 로컬 환경용 설치
        print("🏠 로컬 패키지 설치 중...")
        packages = [
            'numpy', 'opencv-python', 'matplotlib', 'pillow', 'tqdm',
            'PyMuPDF', 'psutil', 'pytesseract', 'requests', 'loguru',
            'huggingface_hub', 'torch', 'ultralytics', 'openai'
        ]
        
        for package in packages:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
                print(f"✅ {package}")
            except subprocess.CalledProcessError:
                print(f"⚠️ {package} 설치 실패")
        
        # DocLayout-YOLO 설치
        if not os.path.exists('DocLayout-YOLO'):
            try:
                subprocess.check_call(['git', 'clone', 'https://github.com/opendatalab/DocLayout-YOLO.git'])
                os.chdir('DocLayout-YOLO')
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-e", "."])
                os.chdir('..')
            except Exception as e:
                print(f"⚠️ DocLayout-YOLO 설치 실패: {e}")

# 설치 실행
ENV = detect_environment()
install_packages()
print("✅ 패키지 설치 완료!")


## 2. 필요한 라이브러리 임포트

In [ ]:
# ============================================
# 📦 라이브러리 Import 및 공통 설정
# ============================================

# 기본 라이브러리
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
import torch
import time
import warnings
import io
import tempfile
import base64
import requests
from PIL import Image
from loguru import logger

# 환경별 tqdm import
if ENV == 'colab':
    from tqdm.notebook import tqdm
    try:
        from google.colab import files
    except ImportError:
        files = None
else:
    from tqdm import tqdm
    files = None

# PDF 처리용
try:
    import fitz  # PyMuPDF
except ImportError:
    if ENV == 'colab':
        !pip install -q PyMuPDF
    else:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "PyMuPDF"])
    import fitz

# DocLayout-YOLO
try:
    from doclayout_yolo import YOLOv10
except ImportError:
    print("❌ DocLayout-YOLO가 설치되지 않았습니다. 이전 셀을 먼저 실행해주세요.")
    YOLOv10 = None

# OCR
import pytesseract

# 경고 메시지 필터링
warnings.filterwarnings('ignore')

# 로깅 설정 (중복 방지)
if len(logger._core.handlers) == 0:  # 핸들러가 없는 경우에만 설정
    logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level: <8} | {name}:{function}:{line} - {message}")
else:
    # 기존 핸들러가 있으면 모두 제거하고 새로 설정
    logger.remove()
    logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level: <8} | {name}:{function}:{line} - {message}")

# ============================================
# 📝 설정 및 상수
# ============================================

class Config:
    """SmartEye 시스템 설정"""
    
    # 지원 파일 형식
    SUPPORTED_IMAGE_FORMATS = {'.jpg', '.jpeg', '.png', '.pdf'}
    
    # OCR 대상 클래스
    OCR_TARGET_CLASSES = [
        'title', 'plain text', 'abandon text',
        'table caption', 'table footnote',
        'isolated formula', 'formula caption'
    ]
    
    # API 대상 클래스
    API_TARGET_CLASSES = ['figure', 'table']
    
    # OCR 설정
    OCR_CONFIG = r'--oem 3 --psm 6'
    
    # 모델 설정
    MODEL_CONFIGS = {
        "docstructbench": {
            "repo_id": "juliozhao/DocLayout-YOLO-DocStructBench",
            "filename": "doclayout_yolo_docstructbench_imgsz1024.pt",
            "imgsz": 1024,
            "conf": 0.25
        },
        "doclaynet_docsynth": {
            "repo_id": "juliozhao/DocLayout-YOLO-DocLayNet-Docsynth300K_pretrained",
            "filename": "doclayout_yolo_doclaynet_imgsz1120_docsynth_pretrain.pt",
            "imgsz": 1536,
            "conf": 0.20
        }
    }

# ============================================
# 🛠️ 유틸리티 함수
# ============================================

def safe_filename(filename, max_length=50):
    """파일명을 안전하게 처리"""
    return "".join(c for c in filename if c.isalnum() or c in '._-')[:max_length]

def ensure_directory(directory):
    """디렉토리 생성"""
    os.makedirs(directory, exist_ok=True)
    return True

def safe_crop_image(img, coordinates):
    """안전한 이미지 크롭"""
    x1, y1, x2, y2 = coordinates
    img_height, img_width = img.shape[:2]
    
    # 좌표 보정
    x1 = max(0, min(x1, img_width - 1))
    y1 = max(0, min(y1, img_height - 1))
    x2 = max(x1 + 1, min(x2, img_width))
    y2 = max(y1 + 1, min(y2, img_height))
    
    return img[y1:y2, x1:x2]

print(f"📍 실행 환경: {ENV.upper()}")
print("🚀 라이브러리 로딩 완료!")

## 3. 사전 훈련된 모델 다운로드 함수 정의

In [ ]:
def download_model(model_choice="docstructbench"):
    """사전 훈련된 DocLayout-YOLO 모델 다운로드"""
    try:
        from huggingface_hub import hf_hub_download
    except ImportError:
        if ENV == 'colab':
            !pip install -q huggingface_hub
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "huggingface_hub"])
        from huggingface_hub import hf_hub_download

    if model_choice not in Config.MODEL_CONFIGS:
        raise ValueError(f"지원하지 않는 모델: {model_choice}")
    
    selected_model = Config.MODEL_CONFIGS[model_choice]

    logger.info(f"모델 다운로드: {selected_model['filename']}")
    filepath = hf_hub_download(
        repo_id=selected_model["repo_id"],
        filename=selected_model["filename"]
    )
    
    logger.info(f"✅ 모델 다운로드 완료: {filepath}")
    return filepath

## 4. 이미지 업로드 및 전처리 컴포넌트

In [ ]:
# ============================================
# 📁 이미지 업로드 및 전처리
# ============================================

def upload_and_prepare_images():
    """환경에 따른 이미지 업로드"""
    if ENV == 'colab':
        return upload_images_colab()
    else:
        return upload_images_local()

def upload_images_colab():
    """Colab 환경용 이미지 업로드"""
    if files is None:
        print("❌ Colab files 모듈을 사용할 수 없습니다.")
        return None
    
    print("📁 지원 형식: JPG, PNG, PDF")
    print("💡 Ctrl+클릭으로 여러 파일 선택 가능")
    
    uploaded = files.upload()
    if not uploaded:
        print("❌ 이미지가 업로드되지 않았습니다.")
        return None
    
    return process_uploaded_files(uploaded)

def upload_images_local():
    """로컬 환경용 이미지 업로드"""
    try:
        import tkinter as tk
        from tkinter import filedialog
        
        root = tk.Tk()
        root.withdraw()
        
        file_paths = filedialog.askopenfilenames(
            title="학습지 이미지 선택",
            filetypes=[
                ("이미지 파일", "*.jpg *.jpeg *.png *.pdf"),
                ("모든 파일", "*.*")
            ]
        )
        
        if not file_paths:
            print("❌ 파일이 선택되지 않았습니다.")
            return None
        
        uploaded = {os.path.basename(path): None for path in file_paths}
        return process_uploaded_files(uploaded, file_paths)
        
    except Exception as e:
        print(f"❌ 로컬 업로드 실패: {e}")
        return None

def process_uploaded_files(uploaded, file_paths=None):
    """업로드된 파일 처리"""
    processed_images = []
    
    for idx, filename in enumerate(uploaded.keys()):
        file_ext = os.path.splitext(filename)[1].lower()
        
        if file_ext not in Config.SUPPORTED_IMAGE_FORMATS:
            print(f"⚠️ 지원하지 않는 형식: {filename}")
            continue
        
        actual_path = file_paths[idx] if file_paths else filename
        
        try:
            if file_ext == '.pdf':
                pdf_images = process_pdf_file(actual_path)
                processed_images.extend(pdf_images)
            else:
                img_info = process_image_file(actual_path)
                if img_info:
                    processed_images.append(img_info)
        except Exception as e:
            print(f"❌ 처리 실패: {filename} - {e}")
    
    print(f"✅ {len(processed_images)}개 이미지 처리 완료")
    display_image_grid(processed_images)
    return processed_images

def process_pdf_file(pdf_path):
    """PDF 파일 처리"""
    pdf_images = []
    
    try:
        pdf_document = fitz.open(pdf_path)
        
        for page_num in range(len(pdf_document)):
            page = pdf_document[page_num]
            
            # 이미지로 변환
            mat = fitz.Matrix(1.5, 1.5)  # zoom factor
            pix = page.get_pixmap(matrix=mat)
            
            img_data = pix.tobytes("png")
            pil_img = Image.open(io.BytesIO(img_data))
            img_array = np.array(pil_img)
            
            if len(img_array.shape) == 3:
                img = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
            else:
                img = cv2.cvtColor(img_array, cv2.COLOR_GRAY2BGR)
            
            # 임시 파일로 저장
            temp_filename = f"temp_pdf_page_{page_num + 1}.png"
            cv2.imwrite(temp_filename, img)
            
            pdf_images.append({
                'original_filename': os.path.basename(pdf_path),
                'processed_filename': temp_filename,
                'page_number': page_num + 1,
                'img_array': img,
                'source_type': 'pdf'
            })
            
        pdf_document.close()
        
    except Exception as e:
        print(f"❌ PDF 처리 실패: {e}")
    
    return pdf_images

def process_image_file(image_path):
    """단일 이미지 파일 처리"""
    try:
        img = cv2.imread(image_path)
        if img is None:
            return None
        
        return {
            'original_filename': os.path.basename(image_path),
            'processed_filename': image_path,
            'page_number': 1,
            'img_array': img,
            'source_type': 'image'
        }
    except Exception as e:
        print(f"❌ 이미지 처리 실패: {e}")
        return None

def display_image_grid(processed_images, max_display=6):
    """이미지 격자 미리보기"""
    if not processed_images:
        return
    
    display_images = processed_images[:max_display]
    num_images = len(display_images)
    
    if num_images == 1:
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        axes = [ax]
    else:
        cols = min(3, num_images)
        rows = (num_images + cols - 1) // cols
        fig, axes = plt.subplots(rows, cols, figsize=(12, 4 * rows))
        if rows == 1:
            axes = axes if isinstance(axes, np.ndarray) else [axes]
        else:
            axes = axes.flatten()
    
    for i, img_info in enumerate(display_images):
        img_display = img_info['img_array']
        
        # 이미지 리사이즈
        max_size = 800
        height, width = img_display.shape[:2]
        if max(height, width) > max_size:
            scale = max_size / max(height, width)
            new_width = int(width * scale)
            new_height = int(height * scale)
            img_display = cv2.resize(img_display, (new_width, new_height))
        
        img_rgb = cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)
        
        ax = axes[i]
        ax.imshow(img_rgb)
        ax.axis('off')
        
        # 제목 설정
        if img_info['source_type'] == 'pdf':
            title = f"{img_info['original_filename']}\n(Page {img_info['page_number']})"
        else:
            title = img_info['original_filename']
        ax.set_title(title, fontsize=10)
    
    # 빈 subplot 숨기기
    for i in range(num_images, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    if len(processed_images) > max_display:
        print(f"📋 총 {len(processed_images)}개 중 {max_display}개만 미리보기 표시")

## 5. 모델 로드 및 설정

In [ ]:
def load_model(model_path):
    """DocLayout-YOLO 모델 로드"""
    try:
        device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        logger.info(f"디바이스: {device}")

        model = YOLOv10(model_path, task='predict')
        model.to(device)

        return model, device

    except Exception as e:
        logger.error(f"모델 로드 실패: {e}")
        raise

## 6. 레이아웃 분석 컴포넌트

In [ ]:
def analyze_layout(model, image_path, img, device='cpu', model_choice="docstructbench"):
    """DocLayout-YOLO를 사용한 레이아웃 분석"""
    logger.info("레이아웃 분석 시작...")

    try:
        model_config = Config.MODEL_CONFIGS.get(model_choice, Config.MODEL_CONFIGS["docstructbench"])
        imgsz = model_config["imgsz"]
        conf = model_config["conf"]

        # 분석 실행
        results = model.predict(
            image_path,
            imgsz=imgsz,
            conf=conf,
            iou=0.45,
            device=device
        )

        if not results or not hasattr(results[0], 'boxes') or results[0].boxes is None:
            logger.warning("감지된 객체가 없습니다.")
            return []

        result = results[0]
        boxes = result.boxes.xyxy.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy()
        confs = result.boxes.conf.cpu().numpy()
        class_names = model.names

        # 레이아웃 정보 구성
        layout_info = []
        img_height, img_width = img.shape[:2]

        for i, (box, cls, conf) in enumerate(zip(boxes, classes, confs)):
            x1, y1, x2, y2 = map(int, box)
            cls_id = int(cls)
            
            try:
                cls_name = class_names[cls_id]
            except (IndexError, KeyError):
                cls_name = f"unknown_{cls_id}"

            # 면적 체크
            area = (x2 - x1) * (y2 - y1)
            if area < 100:  # 최소 면적 임계값
                continue

            # 좌표 보정
            x1 = max(0, min(x1, img_width))
            y1 = max(0, min(y1, img_height))
            x2 = max(x1, min(x2, img_width))
            y2 = max(y1, min(y2, img_height))

            layout_info.append({
                'id': i,
                'class_name': cls_name,
                'confidence': float(conf),
                'box': [x1, y1, x2, y2],
                'width': x2 - x1,
                'height': y2 - y1,
                'area': area
            })

        logger.info(f"✅ {len(layout_info)}개 레이아웃 영역 감지")
        return layout_info

    except Exception as e:
        logger.error(f"레이아웃 분석 실패: {e}")
        raise

## 7. 시각화 컴포넌트

In [ ]:
def visualize_results(img, layout_info, alpha=0.3):
    """레이아웃 분석 결과 시각화"""
    logger.info("결과 시각화 중...")

    try:
        img_result = img.copy()
        overlay = img.copy()

        # 클래스별 색상 생성
        import colorsys
        unique_classes = list(set(layout['class_name'] for layout in layout_info))
        
        class_colors = {}
        for i, cls_name in enumerate(unique_classes):
            h = i / max(1, len(unique_classes))
            r, g, b = colorsys.hsv_to_rgb(h, 0.8, 0.9)
            class_colors[cls_name] = (int(b * 255), int(g * 255), int(r * 255))

        # 바운딩 박스 그리기
        for layout in layout_info:
            x1, y1, x2, y2 = layout['box']
            cls_name = layout['class_name']
            color = class_colors[cls_name]

            # 반투명 채우기
            cv2.rectangle(overlay, (x1, y1), (x2, y2), color, -1)
            
            # 테두리
            cv2.rectangle(img_result, (x1, y1), (x2, y2), color, 2)

            # 라벨
            label = f"{cls_name} ({layout['confidence']:.2f})"
            labelSize, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            y1_label = max(y1, labelSize[1] + 10)
            
            cv2.rectangle(img_result, (x1, y1_label - labelSize[1] - 10),
                         (x1 + labelSize[0], y1_label), color, -1)
            cv2.putText(img_result, label, (x1, y1_label - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

        # 오버레이 적용
        img_result = cv2.addWeighted(overlay, alpha, img_result, 1 - alpha, 0)
        img_rgb = cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB)

        # 시각화
        plt.figure(figsize=(12, 8))
        plt.imshow(img_rgb)
        plt.axis('off')
        plt.title('레이아웃 분석 결과')
        plt.tight_layout()
        plt.show()

        # 클래스별 통계
        class_counts = {}
        for layout in layout_info:
            cls = layout['class_name']
            class_counts[cls] = class_counts.get(cls, 0) + 1
        
        print("\n📊 감지된 클래스:")
        for cls, count in sorted(class_counts.items()):
            print(f"  - {cls}: {count}개")

        return img_result

    except Exception as e:
        logger.error(f"시각화 실패: {e}")
        return img

## 8. 전체 시스템 통합 함수

In [ ]:
def analyze_multiple_worksheets(image_list=None, model_choice="docstructbench"):
    """다중 학습지 분석 파이프라인"""
    start_time = time.time()
    logger.info(f"다중 학습지 분석 시작 - 모델: {model_choice}")
    
    try:
        # 1. 이미지 업로드
        if not image_list:
            image_list = upload_and_prepare_images()
            if not image_list:
                logger.error("이미지 업로드 실패")
                return None
        
        # 2. 모델 로드
        logger.info("모델 다운로드 및 로드 중...")
        model_path = download_model(model_choice)
        model, device = load_model(model_path)
        
        # 3. 결과 디렉토리 설정
        output_dir = "analysis_results"
        ensure_directory(output_dir)
        
        # 4. 각 이미지 처리
        all_results = []
        
        for img_info in tqdm(image_list, desc="이미지 분석"):
            logger.info(f"처리 중: {img_info['original_filename']}")
            
            img = img_info['img_array']
            image_path = img_info['processed_filename']
            
            # 레이아웃 분석
            layout_info = analyze_layout(model, image_path, img, device, model_choice)
            
            # 결과 시각화
            result_img = visualize_results(img, layout_info)
            
            # 결과 저장
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            safe_name = safe_filename(img_info['original_filename'])
            
            if img_info['source_type'] == 'pdf':
                output_path = f"{output_dir}/{safe_name}_p{img_info['page_number']}_{timestamp}.jpg"
            else:
                output_path = f"{output_dir}/{safe_name}_{timestamp}.jpg"
            
            cv2.imwrite(output_path, result_img)
            
            # 결과 구성
            result = {
                'original_filename': img_info['original_filename'],
                'page_number': img_info.get('page_number', 1),
                'source_type': img_info['source_type'],
                'layout_info': layout_info,
                'layout_info_tspm': layout_info.copy(),  # TSPM용
                'output_image_path': output_path,
                'input_image_path': image_path,
                'detected_objects_count': len(layout_info)
            }
            
            all_results.append(result)
            logger.info(f"✅ {img_info['original_filename']} 분석 완료 ({len(layout_info)}개 객체)")
        
        # 5. 결과 정리
        total_time = time.time() - start_time
        successful_results = [r for r in all_results if 'error' not in r]
        
        logger.info(f"✅ 분석 완료 (소요 시간: {total_time:.2f}초)")
        logger.info(f"성공: {len(successful_results)}개")
        
        if successful_results:
            total_layouts = sum(r.get('detected_objects_count', 0) for r in successful_results)
            logger.info(f"총 {total_layouts}개 레이아웃 영역 감지")
            
            # 환경별 결과 다운로드
            if ENV == 'colab' and files:
                download_choice = input("결과 이미지를 다운로드하시겠습니까? (y/n): ")
                if download_choice.lower() == 'y':
                    for result in successful_results:
                        if result.get('output_image_path'):
                            files.download(result['output_image_path'])
                    logger.info("✅ 다운로드 완료!")
            else:
                logger.info(f"결과가 {output_dir} 폴더에 저장되었습니다.")
        
        return all_results
        
    except Exception as e:
        logger.error(f"분석 중 오류 발생: {e}")
        return None
    
    finally:
        # 리소스 정리
        if 'model' in locals():
            del model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

# 단일 이미지 분석 함수 (호환성)
def analyze_worksheet(image_path=None, model_choice="docstructbench"):
    """단일 학습지 분석"""
    if image_path:
        img_info = process_image_file(image_path)
        if img_info:
            results = analyze_multiple_worksheets([img_info], model_choice)
            return results[0] if results else None
    else:
        results = analyze_multiple_worksheets(None, model_choice)
        return results[0] if results and len(results) == 1 else results

## 9. 분석 실행

In [ ]:
# ============================================
# 🚀 LAM 분석 실행
# ============================================

print(f"🚀 {ENV.upper()} 환경용 학습지 분석 시스템")
print("=" * 50)

# 모델 선택
print("사용할 모델을 선택하세요:")
print("1. DocStructBench (학습지/교과서 최적화) - 권장")
print("2. DocLayNet-Docsynth300K (일반 문서 최적화)")

user_choice = input("선택 (1-2, 기본값: 1): ").strip()
model_choice = "doclaynet_docsynth" if user_choice == "2" else "docstructbench"

print(f"\n선택된 모델: {model_choice}")
print(f"지원 형식: JPG, PNG, PDF")
print(f"환경: {ENV.upper()}")

# 분석 실행
analysis_results = analyze_multiple_worksheets(model_choice=model_choice)

# 결과 처리 및 전역 변수 설정
if analysis_results:
    successful_results = [r for r in analysis_results if 'error' not in r]
    
    if successful_results:
        total_images = len(successful_results)
        total_objects = sum(r.get('detected_objects_count', 0) for r in successful_results)
        
        print(f"\n{'='*50}")
        print("📊 LAM 분석 결과")
        print(f"{'='*50}")
        print(f"✅ 처리된 이미지: {total_images}개")
        print(f"🎯 감지된 객체: {total_objects}개")
        
        # 전역 변수 설정 (TSPM용)
        all_analysis_results = analysis_results
        
        # 첫 번째 이미지를 기본값으로 설정
        first_result = successful_results[0]
        image_path = first_result['input_image_path']
        layout_info_tspm = first_result['layout_info_tspm']
        
        # 모든 이미지 정보
        all_images_for_tspm = []
        for result in successful_results:
            all_images_for_tspm.append({
                'image_path': result['input_image_path'],
                'layout_info_tspm': result['layout_info_tspm'],
                'original_filename': result['original_filename'],
                'page_number': result['page_number'],
                'source_type': result['source_type'],
                'result_data': result
            })
        
        print(f"\n✅ TSPM 처리 준비 완료!")
        print(f"   - 처리 대상: {len(all_images_for_tspm)}개 이미지")
        print(f"   - 첫 번째: {first_result['original_filename']}")
        print(f"   - 레이아웃 블록: {len(layout_info_tspm)}개")
        
        print("\n🔄 다음 TSPM 셀을 실행하세요!")
        
    else:
        print("\n❌ 처리된 이미지가 없습니다.")
        all_analysis_results = []
        all_images_for_tspm = []
        
else:
    print("\n❌ 분석이 완료되지 않았습니다.")
    all_analysis_results = []
    all_images_for_tspm = []

print(f"\n LAM 분석 완료!")





# **TSPM (OpenAI Vision API)**

## **Colab 환경 설정 및 필요 라이브러리 설치 컴포넌트**

In [ ]:
# ============================================
# 📦 TSPM 환경 설정 (LAM 통합 후)
# ============================================

print(f"🔧 TSPM 환경 확인... (환경: {ENV})")

# 필수 라이브러리 확인
try:
    import pytesseract
    import requests
    from PIL import Image
    
    print("✅ TSPM 라이브러리 확인 완료!")
    
except ImportError as e:
    print(f"❌ TSPM 라이브러리 import 실패: {e}")
    print("⚠️ LAM 섹션을 먼저 실행해주세요.")

logger.info("✅ TSPM 환경 설정 완료!")

## **라이브러리 import 및 로그 설정 컴포넌트**

In [ ]:
# ============================================
# 📥 TSPM 추가 라이브러리 import
# ============================================

# 텍스트 처리
import textwrap
import re

# OpenAI (선택적)
try:
    import openai
except ImportError:
    print("OpenAI 라이브러리가 설치되지 않았습니다.")
    openai = None

print("✅ TSPM 라이브러리 import 완료!")

In [ ]:
# ============================================
# 🔑 OpenAI API 키 설정
# ============================================

import openai
import os

def setup_openai_api():
    """OpenAI API 키 설정"""
    
    # 1. 환경변수에서 확인
    api_key = os.getenv('OPENAI_API_KEY')
    
    if api_key:
        openai.api_key = api_key
        print("✅ OpenAI API 키가 환경변수에서 로드되었습니다!")
        return True
    
    # 2. Colab Secrets 확인 (가능한 경우)
    try:
        from google.colab import userdata
        api_key = userdata.get('OPENAI_API_KEY')
        if api_key:
            openai.api_key = api_key
            print("✅ OpenAI API 키가 Colab Secrets에서 로드되었습니다!")
            return True
    except Exception:
        pass
    
    # 3. 직접 입력
    print("🔑 OpenAI API 키가 필요합니다.")
    print("   발급: https://platform.openai.com/api-keys")
    
    api_key = input("OpenAI API 키 입력 (sk-로 시작): ").strip()
    
    if api_key and api_key.startswith('sk-'):
        openai.api_key = api_key
        print("✅ OpenAI API 키가 설정되었습니다!")
        return True
    else:
        print("❌ 올바른 API 키를 입력해주세요.")
        return False

# API 키 설정 실행
if setup_openai_api():
    print("🤖 OpenAI Vision API 사용 준비 완료!")
else:
    print("⚠️ Vision API를 사용하려면 올바른 API 키가 필요합니다.")

## **LAM 출력 데이터 로드 컴포넌트**

: LAM에서 필요한 데이터를 TSPM으로 불러오는 컴포넌트.

- Layout Analysis 의 결과 파이션 딕셔너리 값을 가진 layout_info 는 LAM 안에서 지속적으로 덮어쓰여짐. 따라서 Layout 분석 직후의 Key값을 백업변수인 layout_info_tspm 에 저장하고 전역변수화 해서 TSPM으로 불러옴.
- 분석 대상이 되는 학습지 이미지가 담긴 image_path 또한 전역변수화 해서 불러옴.
- 원본이미지를 OpenCV로 로드하여 출력하여 확인 가능함.

In [ ]:
# ============================================
# 📤 LAM 출력 데이터 로드
# ============================================

# LAM에서 넘어온 데이터 확인
try:
    if 'all_images_for_tspm' not in globals():
        logger.warning("⚠️ LAM 분석 결과가 없습니다. LAM을 먼저 실행해주세요.")
        all_images_for_tspm = []
    else:
        logger.info(f"✅ 전체 이미지 데이터 로드: {len(all_images_for_tspm)}개")
        
        # 각 이미지 정보 출력
        for i, img_info in enumerate(all_images_for_tspm):
            filename = img_info['original_filename']
            layout_count = len(img_info['layout_info_tspm'])
            logger.info(f"  {i+1}. {filename} - {layout_count}개 블록")
            
except Exception as e:
    logger.error(f"❌ 데이터 로드 실패: {e}")
    all_images_for_tspm = []

# 기존 호환성 변수들
try:
    if 'layout_info_tspm' not in globals():
        layout_info_tspm = []
    if 'image_path' not in globals():
        image_path = None
        
    logger.info(f"✅ 호환성 변수 확인: layout_info_tspm={len(layout_info_tspm)}개")
except Exception as e:
    logger.error(f"❌ 호환성 변수 설정 실패: {e}")

# 모든 이미지 로드
all_images_loaded = []
if all_images_for_tspm:
    for i, img_info in enumerate(all_images_for_tspm):
        try:
            img_path = img_info['image_path']
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                if img is not None:
                    all_images_loaded.append({
                        'index': i,
                        'original_filename': img_info['original_filename'],
                        'page_number': img_info['page_number'],
                        'source_type': img_info['source_type'],
                        'img_path': img_path,
                        'img': img,
                        'layout_info': img_info['layout_info_tspm']
                    })
                    
        except Exception as e:
            logger.error(f"❌ 이미지 {i+1} 로드 실패: {e}")

# 기본 이미지 설정 (호환성)
if image_path and os.path.exists(image_path):
    img = cv2.imread(image_path)
    if img is not None:
        height, width = img.shape[:2]
        logger.info(f"✅ 기본 이미지 로드: {width}x{height}")
else:
    img = None

print(f"\n🎯 TSPM 처리 준비 완료!")
print(f"   - 총 이미지: {len(all_images_loaded)}개")
print(f"   - 기본 블록: {len(layout_info_tspm)}개")

if len(all_images_loaded) > 0:
    print(f"   - 모든 이미지가 자동으로 처리됩니다!")
else:
    print(f"   - LAM 분석을 먼저 실행해주세요.")

## **OCR 컴포넌트**

### **Tesseract OCR 컴포넌트**

In [ ]:
# ============================================
# 📝 OCR 컴포넌트 (개선된 버전)
# ============================================

import pytesseract
from io import BytesIO
import signal
import time

class TimeoutError(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutError("OCR 처리 시간 초과")

def extract_text_from_region(img, region, timeout=10):
    """영역에서 텍스트 추출 (타임아웃 포함)"""
    try:
        x1, y1, x2, y2 = region
        
        # 좌표 유효성 검사
        if x1 >= x2 or y1 >= y2:
            logger.warning(f"⚠️ 잘못된 좌표: ({x1},{y1}) - ({x2},{y2})")
            return ""
            
        # 이미지 크기 확인
        h, w = img.shape[:2]
        x1, y1, x2, y2 = max(0, x1), max(0, y1), min(w, x2), min(h, y2)
        
        cropped_img = img[y1:y2, x1:x2]
        
        if cropped_img is None or cropped_img.size == 0:
            return ""     
        # 타임아웃 설정
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)
        
        try:
            # Tesseract OCR 수행 (기본 설정)
            text = pytesseract.image_to_string(
                cropped_img, 
                lang='kor+eng',
                config='--psm 6 --oem 3'  # 기본 설정
            )
            return text.strip()
        finally:
            signal.alarm(0)  # 타임아웃 해제
        
    except TimeoutError:
        logger.warning(f"⏰ OCR 타임아웃 (좌표: {region})")
        return ""
    except Exception as e:
        logger.error(f"❌ OCR 실패: {e}")
        return ""

def extract_text_from_blocks(img, layout_info, max_blocks=5):
    """모든 블록에서 텍스트 추출 (제한된 수량)"""
    results = []
    
    print(f"🔍 OCR 시작: {min(len(layout_info), max_blocks)}개 블록 처리...")
    
    for i, block in enumerate(layout_info[:max_blocks]):  # 최대 5개만 처리
        try:
            print(f"   처리 중: {i+1}/{min(len(layout_info), max_blocks)}")
            
            # 블록 정보 파싱
            box = block['box']
            label = block.get('label', 'unknown')
            conf = block.get('conf', 0.0)
            
            # 텍스트 추출 (타임아웃 5초)
            text = extract_text_from_region(img, box, timeout=5)
            
            if text and len(text.strip()) > 0:
                results.append({
                    'block_id': i,
                    'label': label,
                    'confidence': conf,
                    'box': box,
                    'text': text
                })
                print(f"   ✅ 텍스트 발견: {text[:30]}...")
            else:
                print(f"   ⚪ 텍스트 없음")
                
        except Exception as e:
            logger.error(f"❌ 블록 {i} 처리 실패: {e}")
            print(f"   ❌ 블록 {i} 실패")
            
    return results

# Tesseract 설치 확인
def check_tesseract():
    """Tesseract 설치 확인"""
    try:
        version = pytesseract.get_tesseract_version()
        print(f"✅ Tesseract 버전: {version}")
        return True
    except Exception as e:
        print(f"❌ Tesseract 설치 확인 실패: {e}")
        print("   해결 방법:")
        print("   1. sudo apt-get install tesseract-ocr")
        print("   2. sudo apt-get install tesseract-ocr-kor")
        return False

# OCR 테스트 실행
print("🧪 OCR 컴포넌트 테스트 시작...")

# Tesseract 확인
tesseract_ok = check_tesseract()

if tesseract_ok and img is not None and len(layout_info_tspm) > 0:
    try:
        logger.info("📝 OCR 테스트 시작...")
        start_time = time.time()
        
        # 제한된 수의 블록만 처리 (성능상 이유)
        ocr_results = extract_text_from_blocks(img, layout_info_tspm, max_blocks=3)
        
        end_time = time.time()
        print(f"\n✅ OCR 완료: {len(ocr_results)}개 블록에서 텍스트 추출 ({end_time-start_time:.1f}초)")
        
        # 텍스트가 있는 블록만 출력
        text_blocks = [r for r in ocr_results if r['text']]
        if text_blocks:
            print(f"   📝 텍스트 발견: {len(text_blocks)}개 블록")
            for result in text_blocks:
                print(f"   - {result['label']}: {result['text'][:50]}...")
        else:
            print(f"   ⚠️ 추출된 텍스트가 없습니다.")
            print(f"   💡 이미지 품질이나 언어 설정을 확인해보세요.")
            
    except Exception as e:
        logger.error(f"❌ OCR 처리 중 오류: {e}")
        print(f"❌ OCR 처리 실패: {e}")
        
elif not tesseract_ok:
    print("❌ Tesseract이 설치되지 않아 OCR을 건너뜁니다.")
    ocr_results = []
else:
    print("⚠️ 이미지 또는 레이아웃 정보가 없어 OCR을 건너뜁니다.")
    ocr_results = []

print("🎯 OCR 컴포넌트 준비 완료!")

## **AI API 컴포넌트**

### **OpenAI GPT Vision API 컴포넌트**

In [ ]:
# 🤖 통합 Vision API 분석 컴포넌트 (환경 무관)
logger.info("🤖 Vision API 분석 시작...")

# OpenAI API 키 확인 (환경 무관)
import os
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("⚠️ OpenAI API 키가 설정되지 않았습니다.")
    print("💡 환경변수 OPENAI_API_KEY를 설정하거나, API 키를 직접 입력해주세요.")
else:
    # OpenAI 클라이언트 설정
    from openai import OpenAI
    client = OpenAI(api_key=api_key)
    
    print("\n🎯 Vision API 설정:")
    print(f"   - 환경: {ENV.upper()} (API는 환경 무관)")
    print(f"   - 모델: gpt-4-turbo")
    print(f"   - 온도: 0.2 (일관성 향상)")
    print(f"   - 최대 토큰: 800 (상세 분석용)")
    print(f"   - 프롬프트: 클래스별 맞춤형 (Figure/Table 전용)")
    print(f"   - 타겟 클래스: figure, table")
    
    # Figure와 Table 블록 찾기
    figure_blocks = [block for block in layout_info_tspm if block.get('class_name') == 'figure']
    table_blocks = [block for block in layout_info_tspm if block.get('class_name') == 'table']
    all_vision_targets = figure_blocks + table_blocks
    
    print(f"\n📊 분석 대상 블록:")
    print(f"   - Figure 블록: {len(figure_blocks)}개")
    print(f"   - Table 블록: {len(table_blocks)}개")
    print(f"   - 총 분석 대상: {len(all_vision_targets)}개")
    
    if not all_vision_targets:
        print("❌ 분석할 Figure/Table 블록이 없습니다.")
    else:
        print(f"\n🚀 Vision API 분석 시작...")
        
        # 모든 figure와 table 블록에 대해 Vision API 분석 실행
        api_results = []
        
        for i, block in enumerate(all_vision_targets):
            block_id = block.get('id')
            block_class = block.get('class_name')
            print(f"\n   📝 분석 중: {i+1}/{len(all_vision_targets)} - {block_class} (ID: {block_id})")
            
            try:
                # Vision API 분석 수행 (개별 블록)
                result = analyze_blocks_with_vision(
                    [block],  # 개별 블록 분석
                    img_path, 
                    target_classes=['figure', 'table'],
                    client=client
                )
                
                if result:
                    api_results.extend(result)
                    # 상세 결과 출력
                    for res in result:
                        if res.get('description'):
                            print(f"   ✅ 설명 생성 완료!")
                            print(f"   📄 설명 길이: {len(res['description'])}자")
                            # 처음 150자만 미리보기
                            preview = res['description'][:150] + "..." if len(res['description']) > 150 else res['description']
                            print(f"   🔍 미리보기: {preview}")
                        
            except Exception as e:
                logger.error(f"❌ 블록 {block_id} 분석 실패: {e}")
                print(f"   ❌ 분석 실패: {e}")
        
        print(f"\n✅ Vision API 분석 완료: {len(api_results)}개 블록")
        
        # 결과 통계
        figure_results = [r for r in api_results if r.get('class_name') == 'figure']
        table_results = [r for r in api_results if r.get('class_name') == 'table']
        
        print(f"\n📈 분석 결과 통계:")
        print(f"   - Figure 분석 완료: {len(figure_results)}개")
        print(f"   - Table 분석 완료: {len(table_results)}개")
        print(f"   - 총 생성된 설명: {len(api_results)}개")
        
        if api_results:
            avg_length = sum(len(r['description']) for r in api_results) / len(api_results)
            print(f"   - 평균 설명 길이: {avg_length:.0f}자")
            print(f"   - 구조화된 설명: ✅ (단계별 분석)")
            
            print(f"\n📝 생성된 설명 미리보기:")
            for i, result in enumerate(api_results[:3]):  # 처음 3개만 미리보기
                desc_preview = result['description'][:100] + "..." if len(result['description']) > 100 else result['description']
                print(f"   {i+1}. {result['class_name']} (ID: {result['id']}): {desc_preview}")
            
            if len(api_results) > 3:
                print(f"   ... 및 {len(api_results)-3}개 추가 결과")
        
        print(f"\n💾 결과가 'api_results' 변수에 저장되었습니다.")
        print(f"🎉 API 분석 완료!")

# **CIM**

### **OCR 결과 & AI API 결과 객체 출력 컴포넌트**

In [ ]:
# 🤖 TSPM Vision API 함수 정의
import base64
from io import BytesIO
from PIL import Image
import json

def analyze_blocks_with_vision(blocks, image_path, target_classes=['figure', 'table'], client=None):
    """
    블록들을 OpenAI Vision API로 분석
    
    Args:
        blocks: 분석할 블록 리스트
        image_path: 이미지 파일 경로
        target_classes: 분석 대상 클래스 리스트
        client: OpenAI 클라이언트
    
    Returns:
        분석 결과 리스트
    """
    if not client:
        logger.error("OpenAI 클라이언트가 제공되지 않았습니다.")
        return []
    
    results = []
    target_blocks = []
    
    # 타겟 클래스에 해당하는 블록들만 필터링
    for block in blocks:
        # class_name 필드를 우선 확인, 없으면 label 필드 확인
        block_class = block.get('class_name') or block.get('label', 'unknown')
        
        if block_class in target_classes:
            target_blocks.append(block)
            logger.info(f"🎯 타겟 블록 발견: ID {block.get('id')} - {block_class}")
    
    if not target_blocks:
        logger.warning(f"❌ 타겟 클래스 {target_classes}에 해당하는 블록이 없습니다.")
        return []
    
    logger.info(f"🎯 Vision API 분석 대상: {len(target_blocks)}개 블록 ({target_classes})")
    
    # 이미지 로드
    try:
        image = Image.open(image_path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
    except Exception as e:
        logger.error(f"❌ 이미지 로드 실패: {e}")
        return []
    
    # 각 블록별로 Vision API 호출
    for i, block in enumerate(target_blocks):
        try:
            block_id = block.get('id', f'block_{i}')
            block_class = block.get('class_name') or block.get('label', 'unknown')
            
            # 블록 영역 추출
            x1, y1, x2, y2 = block.get('x1', 0), block.get('y1', 0), block.get('x2', 0), block.get('y2', 0)
            
            # 좌표가 유효한지 확인
            if x2 > x1 and y2 > y1:
                # 블록 영역 크롭
                cropped_image = image.crop((x1, y1, x2, y2))
            else:
                # 좌표가 없으면 전체 이미지 사용
                cropped_image = image
            
            # 이미지를 base64로 인코딩
            buffer = BytesIO()
            cropped_image.save(buffer, format='PNG')
            image_data = base64.b64encode(buffer.getvalue()).decode('utf-8')
            
            # 클래스별 맞춤형 프롬프트 설정
            if block_class == 'figure':
                system_prompt = """당신은 시각 장애 아동을 위한 전문 교육 도우미입니다. 
                교육용 학습지의 그림/도표를 분석하여 아이들이 완전히 이해할 수 있도록 
                상세하고 체계적으로 설명해주세요. 교육적 맥락을 고려하여 설명하세요."""
                
                user_prompt = """이 그림을 시각 장애 아동이 완전히 이해할 수 있도록 다음 순서로 자세히 설명해주세요:

                1. **전체 구조**: 그림의 전반적인 구성과 레이아웃
                2. **주요 객체들**: 
                   - 각 객체의 정확한 개수와 이름
                   - 색상, 크기, 모양의 세부 사항
                   - 위치와 배치 관계
                3. **텍스트 요소**: 그림에 포함된 글자, 숫자, 기호들
                4. **교육적 의미**: 이 그림이 가르치려는 개념이나 문제
                5. **학습 활동**: 아동이 이 그림으로 할 수 있는 학습 활동

                명확하고 구체적으로 설명하되, 아동의 이해 수준에 맞게 쉬운 언어를 사용해주세요."""
                
            elif block_class == 'table':
                system_prompt = """당신은 시각 장애 아동을 위한 전문 교육 도우미입니다. 
                교육용 학습지의 표/차트를 분석하여 아이들이 완전히 이해할 수 있도록 
                체계적이고 논리적으로 설명해주세요. 표의 구조와 데이터를 명확히 전달하세요."""
                
                user_prompt = """이 표를 시각 장애 아동이 완전히 이해할 수 있도록 다음 순서로 자세히 설명해주세요:

                1. **표의 구조**: 
                   - 전체 행(가로줄)과 열(세로줄)의 개수
                   - 제목 행/열이 있는지 여부
                   - 표의 전반적인 크기와 형태
                2. **헤더 정보**: 각 열과 행의 제목이나 라벨
                3. **데이터 내용**: 
                   - 각 칸의 정확한 내용 (숫자, 텍스트, 기호 등)
                   - 행별, 열별로 체계적으로 설명
                4. **패턴과 관계**: 데이터 간의 규칙성이나 관련성
                5. **교육적 목적**: 이 표가 가르치려는 개념이나 정보
                
                표를 읽는 순서(왼쪽→오른쪽, 위→아래)를 명확히 하여 설명해주세요."""
                
            else:  # 기타 클래스 (예: chart, diagram 등)
                system_prompt = """당신은 시각 장애 아동을 위한 전문 교육 도우미입니다. 
                교육용 학습지의 시각적 요소를 분석하여 아이들이 완전히 이해할 수 있도록 
                상세하고 체계적으로 설명해주세요."""
                
                user_prompt = f"""이 {block_class}를 시각 장애 아동이 완전히 이해할 수 있도록 자세히 설명해주세요:

                1. **전체 구조**: 전반적인 구성과 레이아웃
                2. **주요 요소들**: 포함된 모든 시각적 요소들
                3. **텍스트와 숫자**: 모든 글자, 숫자, 기호들
                4. **색상과 배치**: 색상 정보와 위치 관계
                5. **교육적 의미**: 학습 목적과 교육적 가치
                
                아동의 이해 수준에 맞는 쉬운 언어로 명확하게 설명해주세요."""
            
            # OpenAI Vision API 호출
            response = client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {
                        "role": "system",
                        "content": system_prompt
                    },
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": user_prompt},
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/png;base64,{image_data}"
                                }
                            }
                        ]
                    }
                ],
                max_tokens=800,  # 더 상세한 설명을 위해 토큰 수 증가
                temperature=0.2
            )
            
            description = response.choices[0].message.content
            
            # 결과 저장
            result = {
                'id': block_id,
                'class_name': block_class,
                'description': description,
                'coordinates': {
                    'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2
                }
            }
            
            results.append(result)
            logger.info(f"✅ API 응답 완료: ID {block_id} - {block_class}")
            
        except Exception as e:
            logger.error(f"❌ 블록 {block_id} 분석 실패: {e}")
            continue
    
    return results

print("✅ analyze_blocks_with_vision 함수 정의 완료!")

### **정렬 & 문제 그룹화 컴포넌트**

### **1차 가공 문서 시각화 컴포넌트**

In [ ]:
# LAM 결과에서 모든 클래스 확인
if 'analysis_results' in globals() and analysis_results:
    # 첫 번째 이미지의 LAM 결과 가져오기
    layout_info_tspm = analysis_results[0]['layout_info']
    print(f"✅ LAM 결과 로드됨: {len(layout_info_tspm)}개 블록")
    
    # 모든 클래스 분포 확인
    from collections import Counter
    class_counts = Counter(block.get('class_name', 'unknown') for block in layout_info_tspm)
    
    print(f"\n📊 전체 클래스 분포:")
    for class_name, count in class_counts.items():
        print(f"   - {class_name}: {count}개")
    
    # Figure와 Table 블록 각각 찾기
    figure_blocks = [block for block in layout_info_tspm if block.get('class_name') == 'figure']
    table_blocks = [block for block in layout_info_tspm if block.get('class_name') == 'table']
    
    print(f"\n🎯 Figure 블록: {len(figure_blocks)}개")
    for i, block in enumerate(figure_blocks):
        print(f"   Figure {i+1}: ID={block.get('id')}")
    
    print(f"\n📋 Table 블록: {len(table_blocks)}개")
    if table_blocks:
        for i, block in enumerate(table_blocks):
            print(f"   Table {i+1}: ID={block.get('id')}")
    else:
        print("   ❌ Table 클래스가 감지되지 않았습니다.")
    
    # Vision API 분석 대상 확인
    vision_targets = figure_blocks + table_blocks
    print(f"\n🤖 Vision API 분석 대상: {len(vision_targets)}개 블록")
    print(f"   - Figure: {len(figure_blocks)}개")
    print(f"   - Table: {len(table_blocks)}개")
        
else:
    print("❌ analysis_results 변수를 찾을 수 없습니다. LAM 분석을 먼저 실행해주세요.")

## 파이선 딕셔너리 JSON 변환 컴포넌트

향후 구현 예정